
# ✈️ Lab 1: Introducción al Caso de Uso "AI Travel Assistant"

## 🎯 Objetivo del proyecto
Crear un **asistente de viajes** inteligente que:

- Encuentre el destino ideal para el usuario usando **RAG** (Retrieval-Augmented Generation).
- Reserve vuelos y hoteles automáticamente.
- Utilice **herramientas (tools)** para ejecutar acciones concretas (buscar vuelos, reservar hoteles).
- Pida **confirmación humana** en pasos críticos (Human-In-The-Loop).

---

## 🗺️ Descripción del Caso de Uso
Imagina un asistente que:

- Personaliza sugerencias de viajes basadas en tus preferencias y viajes pasados.
- Usa **Amazon Bedrock** (modelos LLM de AWS) y el framework open source **LangGraph** de **LangChain**.
- Ofrece una experiencia de planeación de viajes completamente automatizada y adaptativa.

---

## 🔥 Escenario de Laboratorios
Se dividirá en varios Labs, cada uno enfocado en construir una parte del sistema:

### 🧩 Lab 2: Fundamentos de Agentes y Memoria
- Construir agentes usando **LangGraph** y el patrón **ReAct**.
- Introducir **memoria a corto plazo** y **memoria a largo plazo** para diálogos multivuelta (multi-turn conversations).

### 🧠 Lab 3: Planificador de Destinos de Ensueño con RAG
- Personalizar itinerarios usando datos históricos y preferencias de usuario.
- Introducir:
  - **Agentes especializados**.
  - **Human-In-The-Loop** para decisiones importantes.
  - **Multi-Modalidad** (datos no solo de texto, por ejemplo, imágenes).

### 🤝 Lab 4: Colaboración Multi-Agente
- Integrar todos los agentes: sugerencias, reserva de vuelos y hoteles.
- Hacer recomendaciones basadas en **viajeros similares**.
- Permitir reservar, modificar y cancelar todo a través del asistente.

### 🆚 Lab 5 (Opcional): Comparación usando Crew.AI
- Repetir la solución con **Crew.AI** para comparar con LangGraph.

### 📏 Lab 6: Evaluación de Agentes con RAGAS
- Usar **RAGAS** (biblioteca open source) para medir y evaluar la calidad de las respuestas y la interacción entre agentes.

---

## 🛠️ Configuración inicial del entorno

- **Paquetes necesarios**:
  ```bash
  pip install boto3 langchain-aws
  ```

- **Dependencias para LangGraph Studio** (`langgraph.json`):

  ```json
  {
    "dependencies": [
      "boto3>=1.35.87",
      "langchain-aws>=0.2.10",
      "."
    ],
    "graphs": {
      "supervisor": "./src/supervisor_agent/graph.py:graph",
      "flight": "./src/flight_agent/graph.py:graph",
      "hotel": "./src/hotel_agent/graph.py:graph"
    },
    "env": "./.env"
  }
  ```

- **Modelos que se usarán**:
  - **Anthropic Claude 3 Sonnet**
  - **Anthropic Claude 3.5 Sonnet**

- **Servicios AWS involucrados**:
  - **Amazon Bedrock** (para los modelos LLM).
  - **IAM** (permisos para acceder a Bedrock).
  
- **Framework de agentes**:
  - **LangGraph** (orquestación basada en grafos).
  - **LangChain** (infraestructura de agentes y herramientas).

---

## 📈 Diagrama de Flujo General

- Supervisor Agent recibe la solicitud.
- Distribuye tareas a:
  - Destination Agent (para sugerir destinos).
  - Flight Agent (para buscar vuelos).
  - Hotel Agent (para buscar hoteles).
- Todo coordinado y visualizado en **LangGraph Studio**.


In [ ]:
# pip freeze | grep -E '^(langchain==|langchain-aws==|langchain-community==|langchain-text-splitters==|langchainhub==|langgraph==|langgraph-checkpoint==|langgraph-sdk==|langsmith==|pypdf==|ipywidgets==|matplotlib==|faiss-cpu==|boto3==|bedrock==)' > requirements.txt

<img src="images/travel_agent_light.png" width="600" height="400">




# 📚 Dataset: Travel History

## 📖 Descripción del Dataset

En este workshop, trabajaremos con un dataset **sintético** de **historial de viajes**.  
Este dataset será esencial para alimentar al agente con información del usuario que ayudará a:

- **Personalizar las recomendaciones de viaje**.
- **Analizar patrones de viaje pasados**.
- **Simular reservas de vuelos y hoteles**.

---

## 🏷️ Campos del Dataset

| Campo | Descripción |
|:------|:------------|
| **Name** | Nombre completo del individuo. |
| **Current_Location** | Ciudad de residencia actual del individuo. |
| **Age** | Edad del individuo (en años). |
| **Past_Travel_Destinations** | Ciudades previamente visitadas (separadas por comas). |
| **Number_of_Trips** | Número total de viajes realizados. |
| **Flight_Number** | Número de vuelo asociado a cada viaje. |
| **Departure_City** | Ciudad de salida de cada vuelo. |
| **Arrival_City** | Ciudad de llegada de cada vuelo. |
| **Flight_Date** | Fecha en la que se tomó cada vuelo. |

---

## 📂 ¿Para qué lo usaremos en el proyecto?

- El **Destination Agent** podrá consultar el historial de viajes pasados para sugerir nuevos destinos basados en **preferencias anteriores**.
- El **Flight Agent** utilizará información como **Departure City** y **Flight Dates** para buscar y simular vuelos relevantes.
- El **Hotel Agent** recomendará hoteles en ciudades **ya visitadas** o en **nuevos destinos sugeridos**.
- El **Supervisor Agent** integrará todo para crear una experiencia fluida de planeación de viajes.



In [1]:
import pandas as pd

df = pd.read_csv("./data/synthetic_travel_data.csv")
df.head(10)

,Id,Name,Current_Location,Age,Past_Travel_Destinations,Number_of_Trips,Past_Travel_Destinations.1,Number_of_Trips.1,Flight_Number,Departure_City,Arrival_City,Flight_Date
0,24.0,Philip Cook,Zurich,29.0,Ljubljana,1.0,Ljubljana,1.0,Z1823,Zurich,Ljubljana,2024-10-11
1,918.0,Andrew Macdonald,Paris,62.0,"Lisbon, Lisbon, Bergen, Paris, Warsaw",5.0,"Lisbon, Lisbon, Bergen, Paris, Warsaw",5.0,O1142,Paris,Lisbon,2024-10-21
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U7234,Paris,Lisbon,2024-03-23
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H9041,Paris,Bergen,2024-03-25
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y7505,Paris,Paris,2023-09-13
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X8568,Paris,Warsaw,2023-05-17
6,709.0,Jonathan Garcia,Sacramento,43.0,New York,1.0,New York,1.0,P8336,Sacramento,New York,2023-07-10
7,314.0,Wendy Yu,Moscow,34.0,"Prague, Valencia, Reykjavik, London",4.0,"Prague, Valencia, Reykjavik, London",4.0,P6922,Moscow,Prague,2023-01-02
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I1305,Moscow,Valencia,2024-05-26
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S2309,Moscow,Reykjavik,2023-04-14




# 🚀 Amazon Bedrock Model Access

## 📖 Introducción

Amazon Bedrock ofrece acceso a una **variedad de modelos fundacionales (FMs)** que son ideales para construir aplicaciones de **IA generativa**.  
Dependiendo del caso de uso, puedes seleccionar modelos optimizados para:

- **Comprensión de lenguaje natural**.
- **Generación de contenido**.
- **Interacción herramienta-agente**.
- **Planificación de tareas complejas**.

En **flujos de trabajo agentic**, donde los modelos no solo responden sino que también **toman decisiones, invocan herramientas y coordinan acciones autónomas**, elegir el modelo adecuado **es fundamental** para el éxito.

---

## 🧠 Modelos que utilizaremos en este workshop

Para las demostraciones, nos centraremos principalmente en los modelos disponibles en Amazon Bedrock:

| Modelo | Fabricante | Capacidades destacadas |
|:-------|:-----------|:------------------------|
| **Claude 3 (Sonnet y Haiku)** | Anthropic | Razonamiento, generación de texto, excelente para agentic reasoning y flujos de trabajo multi-turn. |
| **Meta Llama 3** | Meta | Optimizado para tareas de lenguaje natural, conversación y tareas de análisis. Ideal para complementar agentes especializados. |
| **Titan Text** | Amazon | Modelos robustos para generación de texto y comprensión general. Útil como soporte para tareas menos complejas. |

🔹 **Nota:** Para explorar todos los modelos disponibles, puedes revisar la documentación oficial: [Supported foundation models in Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)

---

## 🛠️ ¿Por qué estos modelos?

- **Claude 3** es altamente competente en tareas de razonamiento, coordinación de múltiples pasos y manejo de herramientas externas, haciéndolo ideal como modelo para **Supervisor Agent**.
- **Meta Llama 3** puede ser usado para **agentes especializados** como los de **búsqueda de vuelos** o **recomendaciones de hotel**, donde se requieren interacciones rápidas y enfocadas.
- **Titan** puede ser útil como **modelo de respaldo** o para generar mensajes, resúmenes, y otras funciones auxiliares.


In [1]:
import boto3

region = 'us-west-2'
bedrock = boto3.client(
    service_name = 'bedrock-runtime',
    region_name = region,
)



# 🤖 Anthropic's Claude en Amazon Bedrock

## 📦 ¿Qué es el paquete `langchain_aws`?

El paquete **`langchain_aws`** es una herramienta que permite **integrar e interactuar** fácilmente con los modelos de lenguaje de **Amazon Bedrock** dentro del framework **LangChain**.  
Proporciona los siguientes beneficios:

- **Acceso simplificado** a los modelos de Amazon Bedrock.
- **Soporte para flujos de trabajo conversacionales** y **agentes autónomos (agentic workflows)**.

---

## 🛠️ ¿Cómo se usa?

```python
from langchain_aws.chat_models.bedrock import ChatBedrock
```

Esta línea importa la clase **`ChatBedrock`** del módulo `chat_models` dentro del paquete `langchain_aws`.

- **`ChatBedrock`**: Es una clase que permite interactuar directamente con los modelos de **chat** de Amazon Bedrock, facilitando la creación de **aplicaciones de IA conversacional** usando diferentes **modelos fundacionales** disponibles en Bedrock.

---

## 📚 Recursos adicionales

Para más detalles técnicos sobre **Claude** en Amazon Bedrock, puedes consultar la documentación oficial:  
👉 [Anthropic's Claude en Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-claude.html)



<img src="images/converse_api.png" width="800" height="600">


In [2]:
# from langchain_aws.chat_models.bedrock import ChatBedrock
from langchain_aws import ChatBedrock

from rich.console import Console
from rich.text import Text

# Create a console object
console = Console()
modelId = 'us.anthropic.claude-3-haiku-20240307-v1:0'
llm = ChatBedrock(
    model_id=modelId,
    client=bedrock,
    beta_use_converse_api=True
)


response = llm.invoke("I am planning a trip  with my family to Hawai next summer, can you give me a travel iternary for 5 days").content
# Format the text as a rich Text object
formatted_text = Text.from_markup(response)

# Print the formatted response using rich
console.print(formatted_text)

Certainly! Here's a suggested 5-day travel itinerary for your trip to Hawaii:

Day 1: Arrival in Oahu
- Upon arrival at the Honolulu International Airport, pick up your rental car and head to your hotel or resort.
- Spend the afternoon exploring Waikiki Beach, known for its soft, golden sand and vibrant atmosphere.
- Take a stroll along the Waikiki Beach Walk, with its shops, restaurants, and street performers.
- Enjoy a sunset dinner at one of the beachfront restaurants.

Day 2: Oahu's North Shore
- Start your day by driving to the North Shore of Oahu, famous for its world-class surfing beaches.
- Visit the Dole Pineapple Plantation and explore the gardens, train, and pineapple-themed activities.
- Stop at the historic town of Haleiwa and explore the local shops, art galleries, and eateries.
- Spend time at the iconic Banzai Pipeline and Waimea Bay, watching the surfers (or try your hand at it if you're 
an experienced surfer).

Day 3: Pearl Harbor and Honolulu
- Spend the morning at the Pearl Harbor National Memorial, visiting the USS Arizona Memorial, the USS Battleship 
Missouri, and the Pacific Aviation Museum.
- Explore the exhibits and learn about the history of the attack on Pearl Harbor.
- In the afternoon, visit the Iolani Palace, the only official royal residence in the United States.
- Finish the day with a visit to the Bishop Museum, showcasing the history and culture of Hawaii.

Day 4: Kualoa Ranch and Windward Oahu
- Start your day with a guided tour of Kualoa Ranch, a working cattle ranch and film location for many movies and 
TV shows.
- Explore the natural beauty of the ranch, including the stunning Jurassic Valley and Secret Island.
- Drive along the windward coast of Oahu, stopping at the Byodo-In Temple, a replica of a 900-year-old Buddhist 
temple in Japan.
- End the day with a visit to the Kailua Beach, known for its soft, white sand and calm waters.

Day 5: Departure
- Spend the morning relaxing at your hotel or resort, taking in the last moments of your Hawaiian vacation.
- Before your departure, consider visiting the Honolulu Museum of Art or the Ala Moana Center for some last-minute 
shopping and dining.
- Head to the airport in time for your departure flight, carrying memories of your wonderful trip to Hawaii.

Remember, this is just a suggested itinerary, and you can adjust it based on your interests and preferences. Enjoy 
your trip to Hawaii!


# 📊 Comparación de modelos a nivel general (Course-grained model comparison)

## 🧪 Introducción

En esta sección, experimentaremos con varios modelos disponibles en **Amazon Bedrock** y realizaremos una **evaluación de nivel general** sobre una de nuestras tareas de interés.  
Debido a que existen **miles de modelos en el mercado**, es imposible evaluarlos todos de manera exhaustiva.  
Por eso, es **necesario hacer un pre-filtrado** de aquellos que:

- Provienen de **proveedores confiables**.
- Han demostrado **buen desempeño** en distintos **benchmarks**.

Amazon Bedrock facilita este proceso al permitirnos trabajar rápidamente con un conjunto de **proveedores confiables** como:

- **Anthropic**
- **Meta**
- **Mistral**
- **Cohere**
- **AI21Labs**
- **Stability AI**
- **Amazon**

Esto nos permite empezar con una **base sólida** para continuar el proceso de selección de modelos.

---



In [9]:
with open("data/examples.txt", "r") as file:
    examples = file.read().splitlines()

## 🛠️ Evaluación Inicial

En esta etapa, realizamos una **evaluación general** sobre el siguiente modelo:

- **Anthropic: Claude 3 Haiku**

> ⚡ Nota:  
> Para este workshop solo tenemos acceso a un modelo.  
> **Idealmente** en tu propia cuenta podrías probar múltiples modelos para hacer una comparación más completa.

---

## 📋 ¿Cómo realizamos la evaluación?

- Creamos un **dataset curado** con **10 ejemplos**.
- El número inicial de ejemplos **no necesita ser grande**, solo lo suficiente para **cubrir de forma aproximada** los tipos de consultas que esperamos de nuestros usuarios.
- Como este proceso es solo para **tener una idea general**, un conjunto pequeño de ejemplos es aceptable.

---

## 🗺️ Escenario de Evaluación

**Descripción:**

- **Tarea**: Resumir (**summarization**).
- **Contenido**: Destinos de vacaciones (**vacation destinations**).
- **Usuario**: Personas que están planeando viajar (**travelers**).
- **Momento**: Actualmente (**durante el desarrollo**).
- **Idioma**: Inglés.

Los ejemplos iniciales se encuentran en el archivo **`examples.txt`**.

**Expansiones posibles**:  
Podríamos extender nuestra evaluación generando ejemplos equivalentes en otros contextos, por ejemplo:

- Consultas de **personas que no son viajeros**.
- Consultas hechas en **otros idiomas** además del inglés.


In [12]:
import pandas as pd
from langchain_core.messages import HumanMessage, SystemMessage
from rich.console import Console
from rich.text import Text

pd.set_option('display.max_colwidth', None)
# Create a console object
console = Console()


def generate_answers(
    examples: list = [],
    system_prompt: SystemMessage = None
):
    modelIds = [
        'us.anthropic.claude-3-haiku-20240307-v1:0',
    ]
    output = pd.DataFrame({
        'example': [],
        'Claude3Haiku': [],
    })
    for example in examples:
        results = [example]
        for modelId in modelIds:
            messages = [
                system_prompt if system_prompt else SystemMessage(content=""),
                HumanMessage(content=example)
            ]
            llm = ChatBedrock(
                model_id=modelId,
                client=bedrock,
                beta_use_converse_api=True
            )
            resp = llm.invoke(messages).content
            results.append(resp)
        output.loc[len(output)] = results
    return output

In [15]:
one_example = examples
output = generate_answers(one_example)




# 🏁 Conclusión

En este cuaderno, hemos cubierto:

- **Alcance del caso de uso** de los laboratorios en este taller.
- **Conjuntos de datos** que utilizaremos a lo largo de todos los laboratorios.
- **Modelos de Amazon Bedrock** que se emplearán en este workshop.

Ahora, por favor, **procede al siguiente laboratorio** para continuar con el desarrollo de las aplicaciones de AI.


In [14]:
output

,example,Claude3Haiku
0,New York.,"Okay, here are some key facts about New York:\n\n- New York City is the largest city in the United States, with an estimated population of over 8.6 million people as of 2021. It's a global center of finance, arts, media, and culture.\n\n- New York is located in the Northeastern United States, with its southern tip situated on the island of Manhattan in the mouth of the Hudson River. It's bordered by New Jersey and Pennsylvania.\n\n- New York is home to many iconic landmarks and attractions, including the Statue of Liberty, Central Park, the Empire State Building, the 9/11 Memorial, Times Square, Broadway, and more.\n\n- The state of New York has a total population of around 19.3 million people, making it the 4th most populous state in the country. Outside of NYC, other major cities include Buffalo, Rochester, Yonkers, and Syracuse.\n\n- New York has a diverse economy spanning finance, technology, real estate, healthcare, transportation, media, and more. It's considered a global center of business and commerce.\n\n- The weather in New York is humid continental, with warm, humid summers and cold, snowy winters. The state's geography is varied, with the Appalachian Mountains in the west and Long Island on the coast.\n\n- New York is known for its vibrant culture, food scene, performing arts, and diverse population. It's one of the most visited and influential places in the United States.\n\nLet me know if you need any other details about New York!"
1,Synthesize the best places to travel to today.,"Here are some of the best places to travel to today, based on factors like weather, events, and popular destinations:\n\n1. Bali, Indonesia: Known for its stunning beaches, lush tropical landscapes, and rich cultural heritage, Bali is a top destination for travelers seeking a mix of relaxation and adventure.\n\n2. Reykjavík, Iceland: Iceland's capital offers a unique blend of modern urban life and natural wonders, with opportunities to see the Northern Lights, explore glaciers, and soak in geothermal hot springs.\n\n3. Tokyo, Japan: As one of the world's most dynamic and technologically advanced cities, Tokyo offers a captivating blend of traditional culture and cutting-edge modernity, with world-class cuisine, shopping, and entertainment.\n\n4. Marrakech, Morocco: This vibrant city in North Africa is renowned for its bustling souks, historic architecture, and rich culinary traditions, making it a popular destination for cultural immersion and exploration.\n\n5. Amalfi Coast, Italy: The picturesque towns along the Amalfi Coast, such as Positano and Ravello, offer stunning coastal scenery, charming historic streets, and delicious Italian cuisine.\n\n6. Cape Town, South Africa: With its iconic Table Mountain, diverse wildlife, and stunning natural landscapes, Cape Town is a must-visit destination for those seeking outdoor adventures, cultural experiences, and modern city amenities.\n\n7. New York City, USA: As one of the most iconic and vibrant cities in the world, New York offers unparalleled dining, entertainment, shopping, and cultural attractions, making it a perennial favorite for travelers.\n\nRemember to consider factors like your personal interests, budget, and the current weather and events in each destination when deciding where to travel today. It's also important to stay up-to-date on any travel advisories or restrictions."
2,Summarize the environment I would encounter in Bali.,"Bali, an island in Indonesia, is known for its diverse and stunning natural environment. Here's a summary of what you might encounter there:\n\n1. Tropical Climate: Bali has a tropical climate, with warm temperatures throughout the year, typically ranging from 25°C to 35°C. The island experiences a wet and a dry season, with the wet season occurring from October to April.\n\n2. Lush Rainforests: Much of Bali's interior is covered by lush, verdant rainforests, featuring a diverse array of plant and animal life. These f

In [16]:
output.to_csv("data/output.csv")